In [52]:
import pandas as pd
import numpy as np
import string
import re

# Import Data and Clean for BertEmbedding

In [53]:
# from google.colab import drive
# drive.mount('/content/drive')

In [54]:
# !unzip '/content/drive/My Drive/UPenn_Courses/CIS520/Final_Project/data/data_kaggle.zip'

In [148]:
# load data from drive
fake_df = pd.read_csv('../data_kaggle/Fake.csv')
true_df = pd.read_csv('../data_kaggle/True.csv')

In [149]:
# assign labels
fake_df['label'] = 'fake'
true_df['label'] = 'true'

## 0. Check data

In [150]:
# check fake data
fake_df.head(2)

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",fake
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",fake


In [151]:
# check true data
true_df.head(2)

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",true
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",true


In [80]:
true_df.text

0        WASHINGTON (Reuters) - The head of a conservat...
1        WASHINGTON (Reuters) - Transgender people will...
2        WASHINGTON (Reuters) - The special counsel inv...
3        WASHINGTON (Reuters) - Trump campaign adviser ...
4        SEATTLE/WASHINGTON (Reuters) - President Donal...
                               ...                        
21412    BRUSSELS (Reuters) - NATO allies on Tuesday we...
21413    LONDON (Reuters) - LexisNexis, a provider of l...
21414    MINSK (Reuters) - In the shadow of disused Sov...
21415    MOSCOW (Reuters) - Vatican Secretary of State ...
21416    JAKARTA (Reuters) - Indonesia will buy 11 Sukh...
Name: text, Length: 21417, dtype: object

In [81]:
fake_df.text

0        Donald Trump just couldn t wish all Americans ...
1        House Intelligence Committee Chairman Devin Nu...
2        On Friday, it was revealed that former Milwauk...
3        On Christmas day, Donald Trump announced that ...
4        Pope Francis used his annual Christmas Day mes...
                               ...                        
23476    21st Century Wire says As 21WIRE reported earl...
23477    21st Century Wire says It s a familiar theme. ...
23478    Patrick Henningsen  21st Century WireRemember ...
23479    21st Century Wire says Al Jazeera America will...
23480    21st Century Wire says As 21WIRE predicted in ...
Name: text, Length: 23481, dtype: object

## 1. Remove source, credit, (VIDEO) that will affect the prediction

In [152]:
textSplitLen = true_df['text'].apply(lambda x: len(x.split(" - ",1)))

In [153]:
pd.Series.unique(textSplitLen)

array([2, 1])

In [154]:
noSourceId = np.arange(len(textSplitLen))[textSplitLen.values==1]
noSourceId

array([2922, 3488, 3782, 4358, 4465, 5290, 5588, 5761, 5784, 6026, 6184,
       6660, 6823, 7922, 8194, 8195, 8247, 8465, 8481, 8482, 8505, 8506,
       8771, 8970, 9008, 9009, 9307, 9618, 9737])

In [155]:
for i, e in enumerate(noSourceId):
    print(e, true_df.text[e][:50])

2922 The following statements were posted to the verifi
3488 The White House on Wednesday disclosed a group of 
3782 The following statements were posted to the verifi
4358 Neil Gorsuch, President Donald Trump’s appointee t
4465 WASHINGTON The clock began running out this week o
5290 The following statements were posted to the verifi
5588 The following statements were posted to the verifi
5761 The following bullet points are from the U.S. Pres
5784 Federal appeals court judge Neil Gorsuch, the U.S.
6026 The following bullet points are from the U.S. Pres
6184 The following bullet points are from the U.S. Pres
6660 Republican members of Congress are complaining tha
6823 Over the course of the U.S. presidential campaign,
7922 After going through a week reminiscent of Napoleon
8194 The following timeline charts the origin and sprea
8195 Global health officials are racing to better under
8247 U.S. President Barack Obama visited a street marke
8465 ALGONAC, MICH.—Parker Fox drifted out of th

In [156]:
true_df[true_df.text==" "]

,title,text,subject,date,label
8970,Graphic: Supreme Court roundup,,politicsNews,"June 16, 2016",true


In [157]:
# def function to remove photo credits from fake news sources
def remove_photo_credit(text):
    if re.search('Photo by .+|Featured image |Flame image by',text):
        text = re.sub('Photo by (.)+$|Featured image (.)+$|Flame image by (.)+$', '',text)
    return text

# def function to remove video
def remove_video_title(title):
    return re.sub('\(Video\)|(\(|\[)(.)*VIDEO(S)?(.)*(\)|\])|^(.)*VIDEO:','',title)

# def function to remove any particular words
def remove_word(text, word):
    return text.replace(word, '')

In [158]:
# clean to split source with text 
## clean fake
fake_df['cleantitle'] = fake_df['title'].apply(lambda x: remove_video_title(x))
fake_df['cleantext'] = fake_df['text'].apply(lambda x:remove_photo_credit(x))
fake_df['source'] = 'unclear'

## clean true 
true_df['cleantitle'] = true_df['title']
true_df['cleantext'] = true_df['text'].apply(lambda x: x.split(" - ",1)[-1])

def source(x):
    if len(x.split(" - ",1))>1: 
        return x.split(" - ",1)[-2]
    else: 
        return 'unclear'

true_df['source'] = true_df['text'].apply(lambda x: source(x))

## 2. Combine datasets and further clean

In [159]:
path = "../data_intermed/"

In [160]:
# combine datasets
news_df = pd.concat([true_df, fake_df], ignore_index=True)
# news_df = news_df[news_df['text']!= " "]
len(news_df)

44898

In [161]:
# clean to remove punctuation in text and title
# https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
# news_df['puretext'] = news_df['cleantext'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)).lower())
# news_df['puretitle'] = news_df['title'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)).lower())

news_df['puretext_wpunc'] = news_df['cleantext'].apply(lambda x: x.lower())
news_df['puretitle_wpunc'] = news_df['cleantitle'].apply(lambda x: x.lower())

news_df.to_csv(path + "news_bert.csv")

In [162]:
news_df.head(2)

,title,text,subject,date,label,cleantitle,cleantext,source,puretext_wpunc,puretitle_wpunc
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",true,"As U.S. budget fight looms, Republicans flip t...",The head of a conservative Republican faction ...,WASHINGTON (Reuters),the head of a conservative republican faction ...,"as u.s. budget fight looms, republicans flip t..."
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",true,U.S. military to accept transgender recruits o...,Transgender people will be allowed for the fir...,WASHINGTON (Reuters),transgender people will be allowed for the fir...,u.s. military to accept transgender recruits o...


# Contextualized Embedding

https://colab.research.google.com/drive/1RhmL0BqNe52FEbdSyLpkfVuCZxE7b5ke#scrollTo=plr13kC_IAoV

https://huggingface.co/transformers/quickstart.html

In [ ]:
!git clone https://github.com/google-research/bert

In [13]:
!pip install transformers

  Using cached transformers-3.5.1-py3-none-any.whl (1.3 MB)
     |████████████████████████████████| 1.1 MB 3.9 MB/s eta 0:00:01
  Using cached sacremoses-0.0.43.tar.gz (883 kB)
     |████████████████████████████████| 2.0 MB 6.7 MB/s eta 0:00:01
     |████████████████████████████████| 284 kB 3.8 MB/s eta 0:00:01
  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893259 sha256=9b2109ea71a82afe6ce2666e15f76a5db45dde4842fd757406437b3bca268cae
  Stored in directory: /Users/yuchen.zhang/Library/Caches/pip/wheels/69/09/d1/bf058f7d6fa0ecba2ce7c66be3b8d012beb4bf61a6e0c101c0
Successfully built sacremoses


## DistilBert

http://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/

In [30]:
import torch
from transformers import BertTokenizer, BertModel

In [31]:
from transformers import DistilBertTokenizer, DistilBertModel

In [32]:
tokenizer_d = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model_d = DistilBertModel.from_pretrained('distilbert-base-uncased')

### Batch assign to avoid crashing

#### Title (Done)

In [163]:
encoded_input_title = tokenizer_d(news_df['puretitle_wpunc'].tolist(), return_tensors = 'pt', truncation = True, padding = True)

In [164]:
torch.save(encoded_input_title, path + "encoded_input_title.pt")

In [165]:
output_title = np.zeros([len(news_df['puretitle_wpunc']), 768])
output_title.shape

(44898, 768)

In [168]:
with torch.no_grad():
    for i in range(45):
        encoded_input_title_batch = {}
        encoded_input_title_batch['input_ids'] = encoded_input_title['input_ids'][i*1000: min(len(news_df['puretitle_wpunc']), (i+1)*1000)]
        encoded_input_title_batch['attention_mask'] = encoded_input_title['attention_mask'][i*1000: min(len(news_df['puretitle_wpunc']), (i+1)*1000)]
        output_title[i*1000: min(len(news_df['puretitle_wpunc']),(i+1)*1000)] = model_d(**encoded_input_title_batch)[0][:,0,:].numpy()

In [169]:
np.save(path + "distilBert_title", output_title)

In [171]:
output_title.shape

(44898, 768)

#### Article (Done)


In [94]:
encoded_input_text = tokenizer_d(news_df['puretext_wpunc'].tolist(), return_tensors = 'pt', truncation = True, padding = True)

In [95]:
torch.save(encoded_input_text, path + "encoded_input_text.pt")

In [96]:
output_text = np.zeros([len(news_df['puretext_wpunc']), 768])
output_text.shape

(44898, 768)

In [97]:
with torch.no_grad():
      for i in range(len(news_df['puretext_wpunc'])):
            encoded_input_text_batch = {}
            encoded_input_text_batch['input_ids'] = encoded_input_text['input_ids'][i: min(len(news_df['puretext_wpunc']), (i+1))]
            encoded_input_text_batch['attention_mask'] = encoded_input_text['attention_mask'][i: min(len(news_df['puretext_wpunc']), (i+1))]
            # print(model_d(**encoded_input_text_batch)[0][:,0,:].numpy().shape)
            output_text[i: min(len(news_df['puretext_wpunc']),(i+1))] = model_d(**encoded_input_text_batch)[0][:,0,:].numpy()

In [98]:
np.save(path + "distilBert_text", output_text)

In [99]:
output_text.shape

(44898, 768)